In [3]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio as rio
from rasterio.features import geometry_mask
from rasterio.transform import from_bounds
from rasterio.warp import reproject, Resampling
import os
import glob
import rioxarray as rxr
from rasterstats import zonal_stats
from shapely.geometry import mapping
from shapely.geometry import Polygon
from osgeo import ogr

In [4]:
# Ruta a la carpeta que contiene los archivos raster
raster_folder = "D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps"

In [5]:
# Obtener una lista de todos los archivos .nc en la carpeta
raster_files = glob.glob(f"{raster_folder}/*.nc")


In [6]:
raster_files

['D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_abaca.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_agave.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_alfalfa.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_almond.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_aniseetc.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_apple.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_apricot.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_areca.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_artichoke.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_asparagus.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.08_avocado.nc',
 'D:/ROSA/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps\\CROPGRIDSv1.

In [7]:
# Cargar el shapefile de hexágonos
hex_shapefile = "D:/ROSA/hex25.shp"
hexagons = gpd.read_file(hex_shapefile)

In [8]:
hexagons.columns

Index(['fid', 'H_ID', 'geometry'], dtype='object')

In [10]:
# Obtener la proyección del shapefil

shapefile_crs = hexagons.crs
shapefile_crs

<Derived Projected CRS: PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GE ...>
Name: WGS_1984_Web_Mercator_Auxiliary_Sphere
Axis Info [cartesian]:
- [east]: Easting (Meter)
- [north]: Northing (Meter)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [11]:
epsg_code = shapefile_crs.to_epsg()
if epsg_code:
    dst_crs = f"EPSG:{epsg_code}"
else:
    dst_crs = shapefile_crs.to_wkt()

In [12]:
# Inicializar un DataFrame de pandas para almacenar los resultados
results_df = pd.DataFrame()
results_df['H_ID'] = hexagons['H_ID']


In [13]:
hexagons.set_crs('epsg:3857', inplace=True, allow_override=True)

,fid,H_ID,geometry
0,1.0,1,"MULTIPOLYGON (((-8987202.415 -469818.808, -898..."
1,2.0,2,"MULTIPOLYGON (((-8990913.062 -480981.695, -899..."
2,3.0,3,"MULTIPOLYGON (((-8976998.123 -451217.170, -897..."
3,4.0,4,"MULTIPOLYGON (((-8976189.941 -452560.999, -897..."
4,5.0,5,"MULTIPOLYGON (((-8965866.173 -427970.498, -896..."
...,...,...,...
5196,5197.0,5197,"MULTIPOLYGON (((-7049557.421 -2207483.738, -70..."
5197,5198.0,5198,"MULTIPOLYGON (((-7044659.675 -2158861.454, -70..."
5198,5199.0,5199,"MULTIPOLYGON (((-7038113.750 -2313550.758, -70..."
5199,5200.0,5200,"MULTIPOLYGON (((-7037185.999 -2273817.539, -70..."


In [14]:
# Iterar sobre cada archivo .nc
for idx, raster_file in enumerate(raster_files):
    raster = xr.open_dataset(raster_file, engine='netcdf4')

    raster = raster.rio.write_crs("epsg:4326", inplace=True)

    # Clip raster con la geometría del shapefile
    try:
        raster = raster.rio.clip(hexagons.geometry.apply(mapping),
 # This is needed if your GDF is in a diff CRS than the raster data
    hexagons.crs)
        
    except Exception as e:
        print(f"Error al recortar el raster {raster_file}: {e}")
        continue
    
    # Verificar si la variable 'croparea' existe en el dataset
    if 'croparea' in raster.data_vars:
        da = raster['croparea']
    else:
        print(f"'croparea' no encontrada en {raster_file}")
        continue  # Si no se encuentra la banda 'croparea', pasar al siguiente archivo



    # Filtrar los valores mayores a cero

    da = da.where(da > 0)
    print(da)
    arr = da.values
    
    # Calcular las estadísticas zonales
    affine = da.rio.transform()


    stats = zonal_stats(hexagons, arr, affine=affine, stats=['count'], nodata= np.nan)
    
    # Convertir las estadísticas a un DataFrame y agregar la columna del archivo
    stats_df = pd.DataFrame(stats)
    stats_df['file'] = raster_file
    
     # Agregar los resultados al DataFrame general
    all_stats_df = pd.concat([results_df, stats_df], ignore_index=True)
    
    # Guardar el DataFrame actualizado en un archivo CSV
    results_df.to_csv(output_csv, index=False)
    print(f"Resultados de {raster_file} guardados en {output_csv}")

c:\Users\Usuario\anaconda3\lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
